# Libraries

In [ ]:
import numpy as np
from astropy.io import fits
import scipy.ndimage as ndi
import matplotlib.pyplot as mp
from google.colab import drive
import glob

mp.rcParams['figure.figsize'] = (15, 10)

In [ ]:
drive.mount('/content/drive/', force_remount=True)
path = "/content/drive/MyDrive/Colab Notebooks/Enviornment/L6/Telescope Data/Imaging Science/"

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/Enviornment/L6/Telescope Data/Imaging Science/"

# Code

Process the bias and dark frames, then generate a master dark frame for 300s exposure by extrapolating from existing shorter exposure darks. This extrapolation accounts for the increase in dark current with exposure time, allowing estimation of a 300s dark frame even if direct measurements at that exposure are unavailable.


In [ ]:
files = glob.glob(path + 'calib*bias*')  # Find all calibration files that match the pattern 'calib*bias*' (bias frames).
Nfiles = len(files)  # Count the number of bias files found.
print('Number of frames:{}'.format(Nfiles))  # Print the number of bias frames.

hdu = fits.open(files[0])  # Open the first bias frame file.
bias0 = hdu[0].data  # Extract the data from the FITS file (image array).
hdu.close()  # Close the FITS file to free resources.

ny, nx = np.shape(bias0)  # Get the dimensions (ny: rows, nx: columns) of the bias frame.

allbias = np.zeros((Nfiles, ny, nx))  # Initialize an empty array to store all bias frames.

for ind, ff in enumerate(files):
    hdu = fits.open(ff)  # Open each bias frame file.
    allbias[ind, ...] = hdu[0].data  # Store the bias frame data in the 'allbias' array.
    hdu.close()  # Close the FITS file to free resources.

masterbias = np.median(allbias, axis=0)  # Compute the master bias frame by taking the median of all bias frames.
fits.writeto('masterbias.fits', masterbias, overwrite=True)  # Save the master bias frame to a new FITS file.
del allbias  # Delete the 'allbias' array to free memory after use.


In [ ]:
def process_dark_frames(path, exposure_time):
    # Find all dark frames that match the specified exposure time in the directory.
    files = glob.glob(path + f'calib*dark*{exposure_time}*')
    Nfiles = len(files)  # Count the number of dark frames found.
    print(f'Number of {exposure_time}s dark frames: {Nfiles}')  # Output the number of dark frames found.

    if Nfiles > 0:  # If there are any dark frames, process them.
        hdu = fits.open(files[0])  # Open the first dark frame file.
        dark0 = hdu[0].data  # Extract the data (image array) from the FITS file.
        hdu.close()  # Close the FITS file to free resources.

        ny, nx = np.shape(dark0)  # Get the dimensions (ny: rows, nx: columns) of the dark frame.
        all_dark = np.zeros((Nfiles, ny, nx))  # Initialize an array to store all dark frames.

        # Loop over all dark frame files and load them into the 'all_dark' array.
        for ind, ff in enumerate(files):
            hdu = fits.open(ff)
            all_dark[ind, ...] = hdu[0].data  # Store the dark frame data at the corresponding index.
            hdu.close()  # Close each FITS file to free resources.

        # Calculate the master dark frame by taking the median across all frames.
        masterdark = np.median(all_dark, axis=0)
        del all_dark  # Delete the 'all_dark' array to free memory.

        # Save the master dark frame to a FITS file.
        fits.writeto(f'masterdark_{exposure_time}.fits', masterdark, overwrite=True)
        print(f'Master dark frame for {exposure_time}s saved as masterdark_{exposure_time}.fits')

    else:
        # If no dark frames were found, print a message indicating that.
        print(f'No dark frames found for {exposure_time}s.')


In [ ]:
process_dark_frames(path, 60)
process_dark_frames(path, 120)

Instead of taking additional dark frames with 300-second exposures (which consumes more time and resources), you can extrapolate the dark frame using the relationship between dark current and time.

In [ ]:
# Load the master bias frame and the master dark frame for 120s exposure time.
masterbias = fits.getdata('masterbias.fits')
masterdark_120 = fits.getdata('masterdark_120.fits')

# Generate the master dark frame for 300s by extrapolating from the 120s dark frame.
# The equation scales the thermal signal (dark current) from 120s to 300s.
# It assumes the dark current is linear with exposure time:
# (masterdark_120 - masterbias) isolates the thermal signal, scales it by (300/120), and adds back the bias.
masterdark_300 = ((masterdark_120 - masterbias) / 120 * 300) + masterbias

# Save the resulting 300s master dark frame to a FITS file.
fits.writeto('masterdark_300.fits', masterdark_300, overwrite=True)


Now combine the flat frames after removing the bias. Since flats are short exposures, the dark current contribution is negligible. The primary CCD calibration needed here is bias subtraction to remove the readout noise.This ensures that the pixel sensitivity variations in the flat frames are measured accurately without contamination from the bias.

An H-alpha filter is designed to let through only a narrow band of wavelengths around 656.28 nm, blocking out most other wavelengths. This is useful for isolating the light emitted by hydrogen in astronomical objects while reducing interference from other types of light (such as starlight or artificial light pollution).

The OIII (Oxygen-III) filter isolates a narrow wavelength band around 500.7 nm, targeting light emitted by doubly ionized oxygen. This is commonly used to observe emission nebulae, particularly planetary nebulae and supernova remnants, revealing areas where oxygen is excited by intense radiation.

 The SII (Sulfur-II) filter focuses on a narrow band around 672.4 nm, highlighting light emitted by singly ionized sulfur. It helps in identifying regions of ionized sulfur in star-forming areas and supernova remnants.

In [ ]:
def load_and_stack_flats(filter_name, masterbias, path):
    # Load flat frames for the specified filter (OIII, SII, etc.) using the provided filter name.
    files = glob.glob(path + f'flat*{filter_name}*')  # Find all flat files matching the filter name.
    Nfiles = len(files)  # Count the number of flat frames found.
    print(f'Number of {filter_name} flat frames: {Nfiles}')  # Print the number of frames.

    if Nfiles > 0:  # If there are flat frames, proceed with the stacking process.
        hdu = fits.open(files[0])  # Open the first flat frame file.
        flat0 = hdu[0].data  # Extract the data from the FITS file (image array).
        hdu.close()  # Close the FITS file to free resources.

        ny, nx = np.shape(flat0)  # Get the dimensions (ny: rows, nx: columns) of the flat frame.
        all_flats = np.zeros((Nfiles, ny, nx))  # Initialize an array to store all flat frames.

        # Loop over each flat frame, load it, subtract the bias, and store the corrected frame.
        for ind, ff in enumerate(files):
            hdu = fits.open(ff)
            flat = hdu[0].data  # Extract the flat frame data.
            hdu.close()

            # Subtract the master bias from each flat frame to remove bias noise.
            flat_corrected = flat - masterbias
            all_flats[ind, ...] = flat_corrected  # Store the corrected flat frame.

        # Stack the corrected flat frames by taking the median to create the master flat frame.
        master_flat = np.median(all_flats, axis=0)

        # Normalize the master flat by dividing by its median value to ensure uniform illumination correction.
        master_flat /= np.median(master_flat)

        # Save the master flat frame as a FITS file for later use.
        fits.writeto(f'masterflat_{filter_name}.fits', master_flat, overwrite=True)

        return master_flat  # Return the master flat frame for further use.
    else:
        # If no flat frames were found, print a message and return None.
        print(f"No {filter_name} flat frames found.")
        return None


In [ ]:
masterbias = fits.getdata('masterbias.fits')
masterdark_60 = fits.getdata('masterdark_60.fits')
masterdark_120 = fits.getdata('masterdark_120.fits')
masterdark_300 = fits.getdata('masterdark_300.fits')

# Repeat for H filter
master_flat_Halpha = load_and_stack_flats('Halpha', masterbias, path)

# Stack the OIII flat frames to create a master flat
master_flat_OIII = load_and_stack_flats('OIII', masterbias, path)

# Repeat for SII filter
master_flat_SII = load_and_stack_flats('SII', masterbias, path)

# Halpha

In [ ]:
num = ['001', '002', '003', '004', '005', '006']  # List of identifiers corresponding to the science frames for different observations (H-alpha, OIII, SII).
exptime = [60, 60, 120, 120, 300, 300]  # Corresponding exposure times (in seconds) for each science frame. Frames '001' and '002' have 60s exposures, '003' and '004' have 120s, and '005' and '006' have 300s.
Nscience = len(num)  # Number of science frames, which corresponds to the length of the 'num' list.


In [ ]:
filefmt = path + 'NGC956_{}Halpha_{}s.fit'  # Define the file format for the science frames, using placeholders for frame number and exposure time.
halpha_reduced = np.zeros((Nscience, ny, nx))  # Initialize an array to store the calibrated H-alpha frames for all science frames.

# CCD gain (conversion factor from ADU to electrons), given in e-/ADU.
ccd_gain = 0.6  # Replace with your camera's actual gain.

for ind, nn in enumerate(num):  # Loop over all science frames.
    exp_time = exptime[ind]  # Get the exposure time for the current frame.

    # Select the appropriate dark frame based on the exposure time.
    if exp_time == 60:
        dark = masterdark_60
    elif exp_time == 120:
        dark = masterdark_120
    else:
        dark = masterdark_300

    # Open the corresponding H-alpha science frame.
    hdu = fits.open(filefmt.format(num[ind], exp_time))
    science_frame = hdu[0].data  # Extract the science frame data.
    hdu.close()  # Close the FITS file to free resources.

    # Apply bias and dark subtraction
    calibrated_frame = (science_frame - masterbias - dark)

    # Apply flat field correction by dividing by the master flat
    flat_corrected_frame = calibrated_frame / master_flat_Halpha

    # Normalize by gain and exposure time, final units will be e-/s
    halpha_reduced[ind, ...] = (flat_corrected_frame / ccd_gain) / exp_time

# The resulting 'halpha_reduced' array contains the fully calibrated H-alpha science frames in e-/s.


Perform a co-add of the frames using a mean to preserve the signal-to-noise ratio (SNR).
Frames are weighted by the inverse variance to give more importance to less noisy frames.
Variance is proportional to the inverse of the square root of the exposure time, meaning frames with longer exposure times have lower variance and thus higher weights. This ensures that higher-quality frames contribute more to the final co-added result.


In [ ]:
# Perform a weighted average of the H-alpha frames, using weights based on exposure time.
# The weights are calculated as (1 - 1/exptime), giving more importance to longer exposure times.
# This ensures that frames with higher signal-to-noise ratio (SNR) have a greater influence on the co-added result.
halpha_coadd = np.average(halpha_reduced, weights=np.array(exptime), axis=0)

# Optionally save the co-added frame as a FITS file.
hdu = fits.PrimaryHDU(halpha_coadd)
hdu.writeto('/content/COADD1.fits', overwrite=True)


In [ ]:
# Display the co-added H-alpha frame using a color scale between 10 and 50, with the origin set to the lower-left corner for correct orientation.
mp.imshow(halpha_coadd, vmin=10, vmax=50, origin='lower')
# Set the y-axis limits to zoom into the region between pixel rows 2000 and 2080.
mp.ylim(2000, 2080)
# Set the x-axis limits to zoom into the region between pixel columns 2090 and 2170.
mp.xlim(2090, 2170)

Due to significant star movement between exposures, image registration is necessary before co-adding. This is done by selecting a reference star and fitting its profile to find the centroid in each exposure. Once the centroids are determined, the images can be shifted to align the star in the same physical position, ensuring proper alignment of all exposures before co-addition. This prevents misalignment artifacts and improves the final image quality.


In [ ]:
!pip install photutils  # Install the 'photutils' package, used for astronomical image processing, including tasks like centroiding and aperture photometry.

!pip install sep  # Install the 'sep' package, which provides functions for source extraction and photometry, commonly used in astronomical data analysis.


In [ ]:
import photutils.centroids as cent  # Import the centroiding functions from the 'photutils.centroids' module, which provides methods for calculating the centroid of stars and other sources in astronomical images.


The centroid_quadratic method is a technique used to find the centroid (the "center of mass") of a star or object in an astronomical image by fitting a quadratic function to the pixel values.

In [ ]:
xcent = []
ycent = []

# Loop over all science frames to find the star's centroid in each frame
for i in range(Nscience):
    # Calculate the centroid using the 'centroid_quadratic' method on a subregion around the star
    xycent = cent.centroid_quadratic(halpha_reduced[i, 2010:2050, 2110:2150])

    # Append the centroid coordinates adjusted for the full image position
    xcent.append(xycent[0] + 2110)  # Adjust for the x-offset of the subregion
    ycent.append(xycent[1] + 2010)  # Adjust for the y-offset of the subregion

# Convert the lists to NumPy arrays for easier calculations
xcent = np.array(xcent)
ycent = np.array(ycent)

# Calculate the offsets in x and y relative to the first image's centroid
xoff = xcent[0] - xcent
yoff = ycent[0] - ycent


In [ ]:
xoff, yoff

In [ ]:
fig, axes = mp.subplots(nrows=2, ncols=3, figsize=(8,5))  # Create a 2x3 grid of subplots with a figure size of 8x5 inches.

# Loop through each subplot and corresponding image region.
for ind, ax in enumerate(axes.flat):
    try:
        # Display a subregion of the H-alpha image (rows 2010:2050, columns 2110:2150) with pixel intensity limits.
        ax.imshow(halpha_reduced[ind, 2010:2050, 2110:2150], vmin=10, vmax=50, origin='lower')

        # Plot the calculated star centroid in the subregion as a red cross.
        ax.scatter([xcent[ind] - 2110], [ycent[ind] - 2010], marker='+', color='red')

    # If there are fewer images than subplots, pass without error.
    except:
        pass


In [ ]:
# Align the H-alpha frames by shifting them based on the calculated offsets from the centroids.
# The shift function moves the image by (yoff, xoff), ensuring that the star is aligned across all frames.
for i in range(1, Nscience):
    halpha_reduced[i, ...] = ndi.shift(halpha_reduced[i, ...], (yoff[i], xoff[i]))


Interpolating the data with splines can lead to artifacts as splines can overshoot the data, you can try np.roll to shift the data. Roll does not interpolate the input but you are limited to integer pixel shifts

In [ ]:
fig, axes = mp.subplots(nrows=2, ncols=3, figsize=(8, 5))  # Create a 2x3 grid of subplots for displaying image regions, with a figure size of 8x5 inches.

# Loop through each subplot and corresponding image region.
for ind, ax in enumerate(axes.flat):
    try:
        # Display a subregion of the H-alpha image (rows 2010:2050, columns 2110:2150) with pixel intensity limits.
        ax.imshow(halpha_reduced[ind, 2010:2050, 2110:2150], vmin=10, vmax=50, origin='lower')

        # Plot the centroid of the star in the first image as a red cross for reference alignment.
        ax.scatter([xcent[0] - 2110], [ycent[0] - 2010], marker='+', color='red')

    # If there are fewer images than subplots, skip the remaining plots without error.
    except:
        pass


In [ ]:
#Coadd again the images

# Variance is inversely proportional to exposure time
inverse_variance_weights = np.array(exptime)

# Co-add the shifted images using inverse variance weighting
halpha_coadd = np.average(halpha_reduced, weights=inverse_variance_weights, axis=0)

In [ ]:
mp.imshow(halpha_coadd,vmin=-1, vmax=5, origin='lower')
mp.ylim(1000,2000)
mp.xlim(1500,2500)

We now measure the flux in one of the *stars*

To measure the flux of a star in the co-added image, we need to perform aperture photometry. This method involves summing the pixel values within a circular region (the aperture) around the star, while subtracting the background noise from the surrounding region.

Select the star: Identify the star's centroid (which you might already have from earlier centroid calculations).

Define an aperture: Use a circular aperture centered on the star.

Subtract background: Measure the background level using an annulus (a ring around the aperture) and subtract it from the total flux.

Compute flux: Sum the pixel values within the aperture and subtract the estimated background to get the star's total flux.

In [ ]:
from photutils.aperture import aperture_photometry, CircularAnnulus, CircularAperture, ApertureStats
# Import aperture photometry tools from the 'photutils' package. This includes:
# - aperture_photometry: for performing photometry within defined apertures.
# - CircularAnnulus and CircularAperture: for defining circular regions and annuli around stars for photometry.
# - ApertureStats: for computing statistics within the defined apertures.

from astropy.visualization import simple_norm
# Import 'simple_norm' from 'astropy.visualization' to easily normalize image data for better visualization.


In [ ]:
#Define the source position and prepare a simple plot to identify the regions for data and background
positions = [(2132.2,2022.2)]
aperture = CircularAperture(positions, r=12)
annulus_aperture = CircularAnnulus(positions, r_in=15, r_out=20)

norm = simple_norm(halpha_coadd, 'sqrt', percent=99)
mp.imshow(halpha_coadd, norm=norm, interpolation='nearest')
mp.ylim(1980,2060)
mp.xlim(2090,2170)

ap_patches = aperture.plot(color='white', lw=2,label='Photometry aperture')
ann_patches = annulus_aperture.plot(color='red', lw=2,label='Background annulus')

handles = (ap_patches[0], ann_patches[0])
mp.legend(loc=(0.17, 0.05), facecolor='#458989', labelcolor='white',
           handles=handles, prop={'weight': 'bold', 'size': 11})


In [ ]:
phot = aperture_photometry(halpha_coadd, aperture)
print(phot)

In [ ]:
apflux = []

for i in np.arange(1,25):
  aperture = CircularAperture(positions, r=i)
  apflux.append(aperture_photometry(halpha_coadd, aperture)['aperture_sum'])

mp.plot(np.arange(1,25), apflux)
mp.ylabel('Flux')
mp.xlabel('Radius')

#Is this what you can naively expect from the image above? If not, what could be wrong?

You are trying to measure the flux (brightness) of a star in an astronomical image. When doing this, there are two main challenges:

Background Noise: The image contains not just the star’s light but also unwanted light from the background (e.g., from the sky, scattered light, or nearby stars). We need to estimate and subtract this background from the star’s total flux.

Outliers in Background: The background might contain outliers like cosmic rays or noise spikes that could distort the background estimation. These need to be removed or "clipped."

Solution Steps:
Aperture Photometry:

Aperture photometry involves placing a circular region (the aperture) around the star and summing the pixel values inside this aperture to estimate the total flux.
However, this total flux includes both the star's light and the background noise.
Background Estimation with an Annulus:

To estimate the background, we define an annulus (a ring-shaped region) around the star. This annulus is outside the main star’s light but still within the surrounding background.
The pixel values in this annulus represent the background light, which can be used to estimate the background flux inside the aperture.
Sigma Clipping to Handle Outliers:

The background annulus may contain outliers such as bright pixels caused by cosmic rays, hot pixels, or other noise. These outliers would distort the background estimation.
Sigma clipping helps remove those outliers by iteratively rejecting pixels that deviate significantly (in this case, more than 3 standard deviations) from the median background level. This way, we get a cleaner estimate of the background.
Calculate the Star's Net Flux:

Once we know the background level (from the annulus), we subtract the background contribution from the total flux measured in the aperture.
The result is the net flux, which represents the star’s brightness with the background noise subtracted.

In [ ]:
from astropy.stats import SigmaClip

#Redefine apertures for convenience
aperture = CircularAperture(positions, r=12)
annulus_aperture = CircularAnnulus(positions, r_in=15, r_out=20)

#Define a clipping scheme
sigclip = SigmaClip(sigma=3.0, maxiters=10)

#Do the photometry
aper_stats = ApertureStats(halpha_coadd, aperture, sigma_clip=None)
bkg_stats = ApertureStats(halpha_coadd, annulus_aperture, sigma_clip=sigclip)

In [ ]:
#This mean is for one pixel
print(bkg_stats.mean)

In [ ]:
#This value is over N pixels
print(aper_stats.sum)

In [ ]:
#The background needs to be scaled to the right number of pixels
total_bkg = bkg_stats.mean * aper_stats.sum_aper_area.value

print(aper_stats.sum - total_bkg)

In [ ]:
apflux = []
apflux_bkg = []

for i in np.arange(1,25):
  aperture = CircularAperture(positions, r=i)
  apflux.append(aperture_photometry(halpha_coadd, aperture)['aperture_sum'])
  apflux_bkg.append(aperture_photometry(halpha_coadd, aperture)['aperture_sum']-bkg_stats.mean * aperture.area)

mp.plot(np.arange(1,25), apflux)
mp.plot(np.arange(1,25), apflux_bkg)
mp.ylabel('Flux')
mp.xlabel('Radius')

#Which of these two lines do you find to be more correct?